# Package

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import glob as glob
import os

# Set directory

In [7]:
os.chdir('C:\\Users\\wqysb\\Google Drive\\papers\\Au SiO2 100\\Au_SiOx\\Comsol_export\\Ez')
os.getcwd()

'C:\\Users\\wqysb\\Google Drive\\papers\\Au SiO2 100\\Au_SiOx\\Comsol_export\\Ez'

# Import and filter data

In [125]:
def get_file_names():
    return glob.glob('*Ez*.txt')

def read_and_filter_data(yrange, zrange, f):
    data = pd.read_csv(f, skiprows = 9, sep='\s+', names=['y','z','Ez'])
    yfilter = np.logical_and(data.y<=yrange[0], data.y>=yrange[1])
    zfilter = np.logical_and(data.z<=zrange[0], data.z>=zrange[1])
    yzf = np.logical_and(yfilter, zfilter)
    fdata = data[yzf].copy().reset_index(drop=True)
    return fdata

def data_dict(yrange, fnlist):
    dlist = {}
    
    for f in fnlist:
        SiO_t = float(f.split('_')[2])
        zrange = [SiO_t+0.3, SiO_t-0.2]
        print(zrange)
        dlist[SiO_t] = read_and_filter_data(yrange, zrange, f)
    return dlist

In [126]:
file = get_file_names()
dl = data_dict([1.5, -1.5], file)

[15.4, 14.9]
[18.2, 17.7]
[1.3, 0.8]
[20.3, 19.8]
[25.3, 24.8]
[3.6999999999999997, 3.1999999999999997]
[30.3, 29.8]
[35.3, 34.8]
[5.1, 4.6]
[40.3, 39.8]
[7.0, 6.5]
[9.200000000000001, 8.700000000000001]


In [108]:
for v in dl.values():
    print(v.z.min())

0.8003437475847894
3.200347527509251
4.600200334071887
6.50034782953612


# Remove cycle and interpolate

In [127]:
def remove_particle(r, data_dict):
    # make the grid and remove the particle area, return a dictionary of grids with key is the thickness of SiOx
    grid_dict = {}
    for k, v in data_dict.items():
        points = np.transpose([v.y, v.z])
        values = v.Ez.values
        
        y_min, y_max = v.y.min(), v.y.max()
        z_min, z_max = v.z.min(), v.z.max()
        # the range of area
        y0, z0 = 0.0, k + r # coordinate of the center of particle
        
        grid_y0, grid_z0 = np.mgrid[y_min:y_max:3000j, z_min:z_max:500j] # make the grid matrix
        
        # use complex number to help setup the grid mask
        grid = grid_y0 + 1.j * grid_z0
        center = y0 + 1.j * z0

        # Set inside the particle to NaN to remove the particle
        grid_mask = np.less_equal(abs(grid - center), r)
        grid[grid_mask] = np.nan
        grid_y1, grid_z1 = grid.real, grid.imag
        
        grid_z2 = griddata(points, values, (grid_y1, grid_z1), method='linear') # interpolate the Ez in the grid with particle removed
        
        grid_dict[k] = grid_z2
    
    return grid_dict

In [128]:
gl = remove_particle(1.1, dl)

In [145]:
gl[1.0][~np.isnan(gl[1.0])].min()

-11911819896.052116

In [134]:
sorted(gl.keys())

[1.0, 3.4, 4.8, 6.7, 8.9, 15.1, 17.9, 20.0, 25.0, 30.0, 35.0, 40.0]

# Plotting

In [157]:
def plot_Ez(grid_dict):
    num_of_sample = len(grid_dict)
    rows = num_of_sample//3+1
    i = 1
    sorted_key_list = sorted(list(grid_dict))
    vma=grid_dict[1.0][~np.isnan(grid_dict[1.0])].max()
    vmi=grid_dict[1.0][~np.isnan(grid_dict[1.0])].min()*0.3
    plt.figure(figsize=(20,rows*2))
    for k in sorted_key_list:
        plt.subplot(rows, 3, i, title=str(k))
        plt.imshow(np.transpose(gl[k]), extent=[-1.5, 1.5, -0.2, 0.3], origin='lower', vmin=vmi, vmax=vma)
        i=i+1
    plt.show()

In [158]:
plot_Ez(gl)

# Export